***How to Create an Automatic Code Comment Generator using NLP:***

## A propos de notre projet

Bonjour, dans ce projet on *vas* affiner un modèle basé sur RoBERTa qui a été entraîné sur des données de code pour générer automatiquement des commentaires pour le code !


## Données


Le jeu de données https://github.com/xing-hu/DeepCom/blob/master/data.7z du projet DeepCom contient des ressources pour l'entraînement et l'évaluation des modèles de génération de commentaires de code. Il comprend des paires de code source et leurs commentaires correspondants, ce qui permet de développer et de tester des modèles capables de générer automatiquement des commentaires pour le code source. Vous pouvez accéder et télécharger ce jeu de données à partir du repository GitHub de DeepCom.

## CodeBERT
Le modèle préentraîné que nous allons affiner provient de l'incroyable article de la division de recherche de Microsoft intitulé CodeBERT: A Pre-Trained Model for Programming and Natural Languages. Ce modèle a  utilisé le jeu de données  pour générer des commentaires, il l'a utilisé pour enseigner à un modèle basé sur RoBERTa à représenter le code et le langage naturel de manière utile. Cette pratique consistant à apprendre à ces grands modèles de langage à représenter le texte de manière utile est désormais courante, car ces représentations se sont révélées utiles pour affiner ces modèles sur d'autres tâches. L'article sur CodeBERT a montré que ces représentations sont utiles en les affinant pour la recherche de code et la génération de commentaires, exactement ce que nous allons faire ! La différence entre leur tâche de génération de commentaires et la nôtre est que nous effectuerons un peu plus de prétraitement et notre modèle pourra générer des commentaires en ligne pour des extraits de code et pas seulement des commentaires au niveau des méthodes.


Alors, comment CodeBERT apprend-il ces représentations ? Il combine deux objectifs d'entraînement différents qui se sont révélés utiles pour le langage naturel. L'objectif de modélisation de langue masquée (MLM), qui provient de l'article original BERT, et l'objectif de détection de token remplacé (RTD), qui provient de l'article ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators. L'objectif MLM consiste à masquer aléatoirement des parties du texte que nous fournissons au modèle et à demander au modèle de prédire ces parties masquées. L'objectif RTD consiste à remplacer des tokens aléatoires dans le texte et à demander au modèle de déterminer quels tokens ont été remplacés. Cependant, pour rendre la tâche plus difficile pour le modèle, ces tokens remplacés tentent d'être des alternatives plausibles et non juste des mots aléatoires. Le modèle CodeBERT a en fait utilisé un modèle basé sur des n-grammes pour générer ces alternatives, tandis que l'article ELECTRA a utilisé un petit modèle basé sur BERT.


![ELECTRA Pretraining Objective](https://nathancooper.io/i-am-a-nerd/images/electra.png) (From ELECTRA Paper)


Au lieu d'utiliser uniquement le langage naturel pour appliquer ces objectifs d'entraînement, CodeBERT a utilisé le code et les docstrings. Cela a permis au modèle CodeBERT d'apprendre une représentation utile du code qui pourrait être utilisée pour d'autres tâches.


 Importe les bibliothèques nécessaires pour la manipulation des fichiers, l'analyse de données, la visualisation, et le traitement du langage naturel.

In [ ]:
import json
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from typing import List
import seaborn as sns
import re
from typing import Optional
import numpy as np
from collections import Counter
from statistics import mean, median, stdev
from transformers import AutoTokenizer

In [ ]:
pwd


****Affichage du contenu du fichier test****

In [ ]:
filename = '/kaggle/input/code_comments/data/data/test.json'
#chemin actuel du fichier

# Lire et décoder chaque ligne JSON individuellement
data = []
with open(filename) as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Erreur lors du décodage de la ligne : {e}")

# Afficher les 5 premiers éléments du fichier JSON
for i in data[:5]:
    print(i)

****Telechargement des donnees dans une dataFrame Pandas****


In [ ]:
def jsonl_to_dataframe(file_path, columns=['code', 'nl']):
    """Telechargement du JSON lignes file dans pandas DataFrame."""
    df = pd.read_json(file_path, orient='records', lines=True)
    return df[columns]

def get_dfs(path: Path) -> List[pd.DataFrame]:
    """convertir les differents data splits en dataframe pandas"""
    #path est le chemin du folder qui stocke les differents splits de types json il est de type Path
    dfs = []
    for split in ["train.json", "valid.json", "test.json"]:
        file_path = path / split
        if not file_path.exists():
            print(f"File not found: {file_path}")
            dfs.append(pd.DataFrame(columns=['mthd', 'cmt']))  # Créer un DataFrame vide pour ce split
        else:
            df = jsonl_to_dataframe(file_path)
            df = df.rename(columns={'code': 'mthd', 'nl': 'cmt'})
            dfs.append(df)

    return dfs

# Chemin vers les fichiers
path = Path('/kaggle/input/code_comments/data/data')

# Charger les DataFrames
df_trn, df_val, df_tst = get_dfs(path)

#afficher la taille avant l'echantillonage
print(len(df_trn), len(df_val), len(df_tst))

# Échantillonner les DataFrames pour reduire la quantite des donnees originaux dans les dataframes
sample = 0.1
df_trn = df_trn.sample(frac=sample) if not df_trn.empty else df_trn
df_val = df_val.sample(frac=sample) if not df_val.empty else df_val
df_tst = df_tst.sample(frac=sample) if not df_tst.empty else df_tst

# Afficher les tailles des DataFrames échantillonnés
print(len(df_trn), len(df_val), len(df_tst))

****Afficher les 5 premiers lignes du dataframe train****

In [ ]:
df_trn.head()

## DATA Cleaning

****Enlever les caracteres non ASCII des dataframes****

In [ ]:
#data cleaning
# collapse
# From https://stackoverflow.com/a/27084708/5768407
def is_ascii(s):
    '''
    Determines if une chaine donnee contient que les ascii characters

    :param s: the string to check
    :returns: whether or not the given string contains only ascii characters
    '''
    try:
        s.encode(encoding='utf-8').decode('ascii') # encode(encoding='utf-8') : Cette méthode convertit la chaîne s en un format binaire (bytes) en utilisant l'encodage UTF-8
    except UnicodeDecodeError:
        return False
    else:
        return True

df_trn = df_trn[df_trn['mthd'].apply(lambda x: is_ascii(x))]
df_val = df_val[df_val['mthd'].apply(lambda x: is_ascii(x))]
df_tst = df_tst[df_tst['mthd'].apply(lambda x: is_ascii(x))]

df_trn = df_trn[df_trn['cmt'].apply(lambda x: is_ascii(x))]
df_val = df_val[df_val['cmt'].apply(lambda x: is_ascii(x))]
df_tst = df_tst[df_tst['cmt'].apply(lambda x: is_ascii(x))]

len(df_trn), len(df_val), len(df_tst)

On voit bien que la longeur des dataframes n'a pas changer donc on peut dire que ses dernieres ne contiennent pas des caracteres non ascii

****Tester la presence des valeurs manquantes dans le dataframe d'entrainement****

In [ ]:
# Charger le DataFrame
path = Path('/kaggle/input/code_comments/data/data/train.json')
df = jsonl_to_dataframe(path)

# Identifier les valeurs manquantes
missing_values = df.isna()
print("DataFrame des valeurs manquantes :")
print(missing_values)

#la somme des valeurs manquantes par colonne
missing_values_sum = df.isna().sum()
print("\nNombre de valeurs manquantes par colonne :")
print(missing_values_sum)

# le nombre total de valeurs manquantes
total_missing_values = df.isna().sum().sum()
print("\nNombre total de valeurs manquantes :")
print(total_missing_values)

Heureusement notre dataframe d'entrainement ne contient pas des valeurs manquantes

****Graphe pour visualiser les valeurs manquantes****

In [ ]:
# Afficher un heatmap des valeurs manquantes
plt.figure(figsize=(8, 6))
sns.heatmap(df.isna(), cbar=False, cmap='viridis')
plt.title("Heatmap des Valeurs Manquantes")
plt.show()

In [ ]:
def has_code(cmt: str) -> bool:
    '''
    Determinine if the given comment contains the HTML <code> tag

    :param cmt: the comment to check whether it contains the HTML <code> tag
    :returns: whether or not the given comment contains the HTML <code> tag
    '''
    if '<code>' in cmt: return True
    else: return False

df_trn = df_trn[~df_trn['cmt'].apply(lambda x: has_code(x))]
df_val = df_val[~df_val['cmt'].apply(lambda x: has_code(x))]
df_tst = df_tst[~df_tst['cmt'].apply(lambda x: has_code(x))]

len(df_trn), len(df_val), len(df_tst)

 ****Ce code a pour but de nettoyer les commentaires en supprimant toutes les balises HTML présentes dans le texte des commentaires.****

In [ ]:
def clean_html(cmt: str) -> str:
    '''
    Remove any HTML tags from a given comment

    :param cmt: the comment to remove any HTML tags from
    :returns: the comment with any HTML tags removed
    '''
    result = re.sub(r"<.?span[^>]*>|<.?code[^>]*>|<.?p[^>]*>|<.?hr[^>]*>|<.?h[1-3][^>]*>|<.?a[^>]*>|<.?b[^>]*>|<.?blockquote[^>]*>|<.?del[^>]*>|<.?dd[^>]*>|<.?dl[^>]*>|<.?dt[^>]*>|<.?em[^>]*>|<.?i[^>]*>|<.?img[^>]*>|<.?kbd[^>]*>|<.?li[^>]*>|<.?ol[^>]*>|<.?pre[^>]*>|<.?s[^>]*>|<.?sup[^>]*>|<.?sub[^>]*>|<.?strong[^>]*>|<.?strike[^>]*>|<.?ul[^>]*>|<.?br[^>]*>", "", cmt)
    return result

df_trn.cmt = df_trn.cmt.apply(clean_html)
df_val.cmt = df_val.cmt.apply(clean_html)
df_tst.cmt = df_tst.cmt.apply(clean_html)

****Suppression des valeurs manquantes, des espaces doubles et des doublons****

In [ ]:

df_trn = df_trn.applymap(lambda x: ' '.join(x.split()).lower())
df_val = df_val.applymap(lambda x: ' '.join(x.split()).lower())
df_tst = df_tst.applymap(lambda x: ' '.join(x.split()).lower())

df_trn = df_trn[~(df_trn['cmt'] == '')]
df_val = df_val[~(df_val['cmt'] == '')]
df_tst = df_tst[~(df_tst['cmt'] == '')]

df_trn = df_trn[~df_trn['cmt'].duplicated()]
df_val = df_val[~df_val['cmt'].duplicated()]
df_tst = df_tst[~df_tst['cmt'].duplicated()]

len(df_trn), len(df_val), len(df_tst)

**Résumé statistique**

In [ ]:
# Afficher le résumé statistique
print("Résumé statistique :")
print(df.describe())

****Suppression des JavaDocs des commentaires dans le DataFrame, ne laissant que la description.****

In [ ]:
# collapse
from tqdm import tqdm
def remove_jdocs(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Remove the JavaDocs leaving only the description of the comment

    :param df: the pandas dataframe to remove the JavaDocs from
    :returns: a new pandas dataframe with the JavaDocs removed
    '''
    methods = []
    comments = []
    for i, row in tqdm(list(df.iterrows())):
        comment = row["cmt"]
        comment = re.sub("([\{\[]).*?([\)\}])", '', comment)


        cleaned = []
        for line in comment.split('\n'):
            if "@" in line: break
            cleaned.append(line)
        comments.append('\n'.join(cleaned))
        methods.append(row["mthd"])
    new_df = pd.DataFrame(zip(methods, comments), columns = ["mthd", "cmt"])

    return new_df

df_trn = remove_jdocs(df_trn);
df_val = remove_jdocs(df_val);
df_tst = remove_jdocs(df_tst);

Voici le resultat final comment apparait le dataset

In [ ]:
df_trn.head()

In [ ]:
len(df_trn)

## **Data Exploring**

 Fonction pour obtenir le nombre de chaque token dans une colonne de DataFrame, puis calcul de la distribution des longueurs de tokens et des limites maximales basées sur le 95e percentile.

In [ ]:
def get_counter(df: pd.DataFrame, tokenizer: AutoTokenizer, col: str) -> Counter:
    '''
    Get the counts for each token in a given pandas dataframe column

    :param df: Un dataframe de type Pandas
    :param tokenizer: Un tokenizer de type Autotokenizer
    dataframe
    :param col: Une chaine de type string indiquant la colonne du dataframe a analyser
    :returns: Le nombre des occurences de chaque token dans la colonne specifiee
       '''
    toks = []
    for i, row in df.iterrows():
      #df.iterrows() retourne un générateur qui produit des paires (index de la ligne, données de la ligne sous forme de série pandas).
        toks.extend(tokenizer.tokenize(row[col]))

    cnt = Counter()
    for tok in toks:
        cnt[tok] += 1
    return cnt

tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base') #Chargement d'un tokenizer pré-entraîné basé sur le modèle microsoft/codebert-base de Hugging Face.
mthd_cnt = get_counter(df_trn, tokenizer, 'mthd')
cmt_cnt = get_counter(df_trn, tokenizer, 'cmt')
mthd_lens = df_trn.mthd.apply(lambda x: len(tokenizer.tokenize(x))).values #Cette méthode (values) extrait les données de la série pandas sous forme de tableau NumPy (array).
cmt_lens = df_trn.cmt.apply(lambda x: len(tokenizer.tokenize(x))).values
max_mthd_len = int(np.quantile(mthd_lens, 0.95)) #Calcul des longueurs maximales à 95% (quantiles) c'est-à-dire la valeur en dessous de laquelle se situent 95% des longueurs de tokens
max_cmt_len = int(np.quantile(cmt_lens, 0.95))

****Cette fonction trace un graphique à barres des tokens les plus fréquents, en affichant jusqu'à top_k tokens les plus courants.****

In [ ]:
# collapse
def plot_counts(counts:Counter, top_k: Optional[int] = 30):
    '''
    :param counts: conteur des tokens et de leurs frequences
    :param top_k: le nombre de token a afficher dans le plot
    '''
    labels, values = zip(*counts.most_common()[:top_k])

    indexes = np.arange(len(labels))
    width = 1
    plt.figure(num=None, figsize=(22, 4), dpi=60, facecolor='w', edgecolor='k')
    plt.bar(indexes, values, width)
    plt.xticks(indexes + width * 0.5, labels)
    plt.show()

In [ ]:
plot_counts(mthd_cnt, top_k = 30)
plot_counts(cmt_cnt, top_k = 30)

****les statistiques descriptives (moyenne, médiane, écart type, min, max, 25e et 75e percentiles)****

In [ ]:
# Calcul des statistiques descriptives pour les longueurs des tokens
def descriptive_stats(lengths):
    return {
        'mean': np.mean(lengths), #moyenne
        'median': np.median(lengths), #mediane
        'std_dev': np.std(lengths), #ecart_type
        'min': np.min(lengths),
        'max': np.max(lengths),
        '25th_percentile': np.percentile(lengths, 25), #premier quartile (Q1)
        '75th_percentile': np.percentile(lengths, 75) #troisième quartile (Q3)
    }

mthd_stats = descriptive_stats(mthd_lens)
cmt_stats = descriptive_stats(cmt_lens)

print("Method Lengths Statistics:", mthd_stats)
print("Comment Lengths Statistics:", cmt_stats)


In [ ]:
print("Top Tokens in Methods:", mthd_cnt.most_common(30))
print("Top Tokens in Comments:", cmt_cnt.most_common(30))

****Création d'un histogramme pour comparer la distribution des longueurs des tokens des méthodes et des commentaires****

In [ ]:
# Histogramme
plt.hist(mthd_lens, bins=50, alpha=0.7, label='Method Lengths')
plt.hist(cmt_lens, bins=50, alpha=0.7, label='Comment Lengths')
plt.legend()
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.title('Distribution of Token Lengths')
plt.show()

# Box Plot
plt.boxplot([mthd_lens, cmt_lens], labels=['Methods', 'Comments'])
plt.ylabel('Number of Tokens')
plt.title('Box Plot of Token Lengths')
plt.show()

**Remarque :**

---

On observe que la plupart des méthodes et des commentaires ont une longueur très courte (nombre de tokens faible), ce qui est représenté par la grande barre à gauche du graphique (proche de 0).
Pour le box plot :
La ligne horizontale à l'intérieur de chaque boîte représente la médiane du nombre de tokens pour les méthodes et les commentaires.Les boîtes représentent les quartiles, c'est-à-dire que 50 % des données se trouvent à l'intérieur de la boîte.En fin on peut dire que le Box plot montre clairement que les méthodes de code ont une variabilité de longueur beaucoup plus grande que les commentaires, avec plusieurs méthodes étant des outliers très longs.

****Calcul de la similarité cosinus entre les commentaires en utilisant TF-IDF pour transformer le texte en vecteurs****

In [ ]:
"""on a eleminer ce code a cause de la grande quantite de donnees car cela demande une grande puissance de calcule"""
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import seaborn as sns

# # Calcul de la similarité cosinus
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df_trn['cmt'])
# similarity_matrix = cosine_similarity(X)

# # Affichage de la matrice de similarité
# sns.heatmap(similarity_matrix, cmap='coolwarm')
# plt.title('Cosine Similarity Matrix')
# plt.show()

## **Model Training**

--> Telechargment de quelques modules necessaires pour l'entrainement

In [ ]:
!pip install torch datasets

In [ ]:
!mkdir java

In [ ]:
!cp  /kaggle/input/fichierprojet/projet/model.py /kaggle/working/

In [ ]:
!cp  /kaggle/input/fichierprojet/projet/bleu.py /kaggle/working/

In [ ]:
!cp  /kaggle/input/fichierprojet/projet/run.py /kaggle/working/

In [ ]:
import json

df_trn['code_tokens'] = df_trn.mthd.apply(lambda x: x.split()) #Ajouter une nouvelle colonne des tokens au dataframe
df_trn['docstring_tokens'] = df_trn.cmt.apply(lambda x: x.split())
with open('java/train.jsonl','w') as f:
    for _, row in df_trn.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n') #Convertir les row en dictionnaire puis pour l'ecriture dans le fichier json il faut avoir un format chaine de caractere c'est le role de dump

df_val['code_tokens'] = df_val.mthd.apply(lambda x: x.split())
df_val['docstring_tokens'] = df_val.cmt.apply(lambda x: x.split())
with open('java/valid.jsonl','w') as f:
    for _, row in df_val.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

df_tst['code_tokens'] = df_tst.mthd.apply(lambda x: x.split())
df_tst['docstring_tokens'] = df_tst.cmt.apply(lambda x: x.split())
with open('java/test.jsonl','w') as f:
    for _, row in df_tst.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

L'objectif de ce code est de préparer les données pour un modèle en créant des tokens pour le code source et les commentaires, puis les sauvegarder au format .jsonl pour une utilisation ultérieure.

In [ ]:
!git clone https://github.com/microsoft/CodeXGLUE.git
!cd CodeXGLUE

Le dépôt CodeXGLUE de Microsoft contient des ressources, des codes, et des jeux de données pour diverses tâches liées au traitement automatique du langage naturel (NLP) et à la programmation assistée par l'intelligence artificielle.(Optionnel)

In [ ]:
lang = 'java' # programming language
lr = 5e-5
batch_size = 8 # change depending on the GPU Colab gives you
beam_size = 10
source_length = 256
target_length = 50
data_dir = '.'
output_dir = f'model/{lang}'
train_file = f'{data_dir}/{lang}/train.jsonl'
dev_file = f'{data_dir}/{lang}/valid.jsonl'
epochs = 10
pretrained_model = 'microsoft/codebert-base'

! python /kaggle/working/run.py \
    --do_train \
    --do_eval \
    --do_lower_case \
    --model_type roberta \
    --model_name_or_path {pretrained_model} \
    --train_filename {train_file} \
    --dev_filename {dev_file} \
    --output_dir {output_dir} \
    --max_source_length {source_length} \
    --max_target_length {target_length} \
    --beam_size {beam_size} \
    --train_batch_size {batch_size} \
    --eval_batch_size {batch_size} \
    --learning_rate {lr} \
    --num_train_epochs {epochs}

L'objectif de ce code est de fine-tuner le modèle CodeBERT(de Hugging face Transformers) sur un ensemble de données spécifiques

In [ ]:
batch_size=64
dev_file=f"{data_dir}/{lang}/valid.jsonl"
test_file=f"{data_dir}/{lang}/test.jsonl"
test_model=f"{output_dir}/checkpoint-best-bleu/pytorch_model.bin" #checkpoint for test

! python /kaggle/working/run.py \
    --do_test \
    --model_type roberta \
    --model_name_or_path microsoft/codebert-base \
    --load_model_path {test_model} \
    --dev_filename {dev_file} \
    --test_filename {test_file} \
    --output_dir {output_dir} \
    --max_source_length {source_length} \
    --max_target_length {target_length} \
    --beam_size {beam_size} \
    --eval_batch_size {batch_size}

L'objectif de ce code est de tester le modèle CodeBERT qui a été entraîné, en utilisant un ensemble de données de test pour évaluer sa performance.

In [ ]:
import re

def detect_classes(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    class_pattern = re.compile(r'class\s+(\w+)\s*\(.*?\):')
    classes = class_pattern.findall(content)

    return classes

file_path = './model.py'
classes = detect_classes(file_path)
print("Classes found:", classes)

In [ ]:
from model import Seq2Seq

In [ ]:
# collapse
import torch

import torch.nn as nn

from transformers import RobertaConfig, RobertaModel

config = RobertaConfig.from_pretrained(pretrained_model)
encoder = RobertaModel.from_pretrained(pretrained_model, config = config)
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model = Seq2Seq(encoder = encoder,decoder = decoder,config=config,
                beam_size=beam_size,max_length=target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
model.load_state_dict(torch.load(Path(output_dir)/"checkpoint-last/pytorch_model.bin"))
model.to('cuda')

Ce code initialise et configure le modèle Seq2Seq basé sur Roberta pour une tâche de génération de séquence. Il est constitué d'un encodeur **Roberta** pré-entraîné et d'un décodeur **Transformer**. Les poids du modèle sont chargés à partir d'un checkpoint sauvegardé, et le modèle est transféré sur le GPU pour exécuter des inférences ou pour une nouvelle phase de test ou d'entraînement.

In [ ]:
idx = 0

In [ ]:
# collapse
from run import convert_examples_to_features, Example

class Args:
    max_source_length = source_length
    max_target_length = target_length

args = Args()

def get_preds(df: pd.DataFrame):
    ps = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        examples = [
            Example(idx, source = row.mthd, target = row.cmt)
        ]
        eval_features = convert_examples_to_features(
            examples, tokenizer, args, stage='test'
        )
        source_ids = torch.tensor(eval_features[0].source_ids, dtype = torch.long).unsqueeze(0).to('cuda')
        source_mask = torch.tensor(eval_features[0].source_mask, dtype = torch.long).unsqueeze(0).to('cuda')

        with torch.no_grad():
            preds = model(source_ids = source_ids, source_mask = source_mask)
            for pred in preds:
                t = pred[0].cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                ps.append(text)

    return ps

La fonction get_preds sert à générer des prédictions de texte à partir des méthodes présentes dans un DataFrame. Chaque méthode est transformée en séquence d'IDs, passée à travers le modèle **Seq2Seq** pour obtenir une prédiction, qui est ensuite décodée en texte. Ce texte est ensuite collecté et renvoyé sous forme de liste de prédictions.

In [ ]:
""" ce code prend les 10 premières lignes du DataFrame, génère des commentaires pour chacune en utilisant notre modèle"""
df_val = df_val.reset_index()
preds = get_preds(df_val.head(10))
for idx, row in df_val.head(10).iterrows():
    print('Code:', row.mthd)
    print('Original Comment:', row.cmt)
    print('Generated Comment:', preds[idx])
    print('='*40)

In [ ]:
def get_preds_losses(df: pd.DataFrame):
    ps = []
    losses = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        examples = [
            Example(idx, source = row.mthd, target = row.cmt)
        ]
        eval_features = convert_examples_to_features(
            examples, tokenizer, args, stage='test'
        )
        source_ids = torch.tensor([f.source_ids for f in eval_features], dtype = torch.long).to('cuda')
        source_mask = torch.tensor([f.source_mask for f in eval_features], dtype = torch.long).to('cuda')
        target_ids = torch.tensor([f.target_ids for f in eval_features], dtype = torch.long).to('cuda')
        target_mask = torch.tensor([f.target_mask for f in eval_features], dtype = torch.long).to('cuda')

        with torch.no_grad():
            _, loss, _ = model(
                source_ids = source_ids, source_mask = source_mask,
                target_ids = target_ids, target_mask = target_mask
            )
            preds = model(source_ids = source_ids, source_mask = source_mask)
            for pred in preds:
                t = pred[0].cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                ps.append(text)
                losses.append(loss.item())

    return ps, losses

La fonction get_preds_losses traite chaque ligne d'un DataFrame pour générer des prédictions et calculer les pertes en utilisant le modèle

In [ ]:
df_head = df_val.copy()
ps, losses = get_preds_losses(df_head)
df_head['pred'] = ps
df_head['loss'] = losses
df_sorted_losses = df_head.sort_values('loss', ascending = False)

for _, row in df_sorted_losses.head(10).iterrows():
    print('Code:', row.mthd)
    print('Original Comment:', row.cmt)
    print('Generated Comment:', row.pred)
    print(row.loss)
    print('='*40)

Ce code génère des prédictions et calcule des pertes pour chaque exemple dans le DataFrame df_val. Ensuite, il ajoute ces résultats au DataFrame, trie les exemples par ordre décroissant des pertes, et affiche les 10 exemples avec les pertes les plus élevées

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk

nltk.download('punkt')  # Téléchargez les ressources nécessaires pour le tokenization

def calculate_bleu(reference, hypothesis):
    if len(hypothesis) == 0:  # Vérifie si l'hypothèse est vide
        return 0.0
    smoothie = SmoothingFunction().method4
    try:
        score = sentence_bleu([reference], hypothesis, smoothing_function=smoothie)
    except ZeroDivisionError:  # Catch division by zero errors
        score = 0.0
    return score

df_head['bleu'] = df_head.apply(lambda row: calculate_bleu(row.cmt.split(), row.pred.split()), axis=1)

print(f"Score BLEU moyen : {df_head['bleu'].mean()}")

Ce code a pour objectif de calculer le score **BLEU** (Bilingual Evaluation Understudy) pour évaluer la qualité des commentaires générés par votre modèle par rapport aux commentaires de référence

**NB:**
un score BLEU seul ne donne pas une image complète de la qualité du modèle, car il ne prend pas en compte la qualité sémantique ou la pertinence des commentaires générés.

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Initialiser le calculateur ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge(reference, hypothesis):
    scores = scorer.score(reference, hypothesis)
    return scores

# Appliquer le calcul ROUGE sur chaque ligne du DataFrame
df_head['rouge_scores'] = df_head.apply(lambda row: calculate_rouge(row.cmt, row.pred), axis=1)

# Extraire les scores ROUGE-1, ROUGE-2, et ROUGE-L
df_head['rouge1'] = df_head['rouge_scores'].apply(lambda x: x['rouge1'].fmeasure)
df_head['rouge2'] = df_head['rouge_scores'].apply(lambda x: x['rouge2'].fmeasure)
df_head['rougeL'] = df_head['rouge_scores'].apply(lambda x: x['rougeL'].fmeasure)

# Afficher les scores moyens
print(f"Score ROUGE-1 moyen : {df_head['rouge1'].mean()}")
print(f"Score ROUGE-2 moyen : {df_head['rouge2'].mean()}")
print(f"Score ROUGE-L moyen : {df_head['rougeL'].mean()}")


**Interpretation des resultats**:
---
**ROUGE-1 moyen (0.34) :** Ce score mesure la correspondance des unigrams (mots individuels) entre le texte de référence et le texte généré. Un score de 0.34 indique qu'environ 34% des unigrams du texte généré se retrouvent dans le texte de référence. Ce score est relativement modéré, suggérant une certaine similarité au niveau des mots individuels.

**ROUGE-2 moyen (0.15) :** Ce score mesure la correspondance des bigrams (paires de mots consécutifs) entre le texte de référence et le texte généré. Un score de 0.15 est plus bas que celui de ROUGE-1, ce qui est habituel, car la correspondance des bigrams est plus difficile à obtenir. Ce score indique que seulement 15% des bigrams correspondent, ce qui peut signifier que la structure des phrases diffère entre le texte généré et le texte de référence.

**ROUGE-L moyen (0.33) :** Ce score mesure la correspondance de la plus longue sous-séquence commune (LCS) entre le texte de référence et le texte généré. Un score de 0.33 est comparable à ROUGE-1, ce qui suggère que la séquence générale des mots dans le texte généré est similaire à celle du texte de référence, mais sans correspondance parfaite.

## **Fin**

Ce projet a porté sur La Generation Automatique des Code JAVA. Tout au long du projet, nous avons développé et évalué un modèle de génération automatique de commentaires en utilisant Le fameux CodeBert en passant par plusieurs etapes essentiels a savoir de Data cleaning ,Data processing et L'entrainnement du modele.